In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd
import os
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
import requests as req
from bs4 import BeautifulSoup
import re
import datetime
from datetime import date
from os import listdir
import glob

In [ ]:
lista_keywords = ['Negro de mierda','Mantero','Extranjeros ilegales','panchito','panchitos','maricón','Sudaca','espaldas mojadas','indio','trasvesti','inmigrantes ilegales','Moro','Moros','Travelo','Paki','Bujarra','sidoso','negrata','Bollera','mariposon','desviado','mariquita','sarasa','machorra','tortillera','comebollos','machurrona','Africano','Latino','virus chino','Homosexual','machupichu','Tiraflechas','afeminado','negrata','indígenas','Sin papeles']

In [ ]:
def url_extractor(lista):
    options = webdriver.ChromeOptions()
    options.add_argument('disable-infobars')
    for keyword in lista:
        #Apertura de página con selenium
        listing_links = []
        link = 'https://eltorotv.com/?s='+ keyword
        
        print(keyword)
        
        driver = webdriver.Chrome(executable_path="./drivers/chromedriver")
        driver.get(link)
        time.sleep(1)
        #Código para evitar que entren noticias no relacionadas.
        resultados = driver.find_elements_by_xpath('//*[@id="main"]/section/header/h1')
    
        status = 'se han obtenido resultados'
        for i in resultados:
            if i.text == 'No hay resultados para: '+ keyword:
                status = 'sin resultados'
            
           
        #Extracción de links con selenium.
        try:
            for i in link:
                links = driver.find_elements_by_css_selector('.post-link')
                for link in links:
                    listing_links.append(str(link.get_attribute('href')))
                time.sleep(1)
                actions = ActionChains(driver) 
                #Scroll para qevitar banners
                for _ in range(6):
                    actions.send_keys(Keys.PAGE_DOWN).perform()
                time.sleep(1)
                driver.find_element_by_xpath("//*[text()='Entradas anteriores']").click()
                driver.close()
        except:
           
            #creación de dataframe con los datos generados
            listing_links = list(dict.fromkeys(listing_links))
            df = pd.DataFrame(listing_links, columns=['link'])
            df['keyword'] = keyword
            df['Status'] = status

            #guardado de archivos, con el nombre de su keyword
            nombre_archivo = keyword+ '_' + status + '.csv'
            path = './news_urls/el_gato_al_agua_urls'
            archivo_de_salida = os.path.join(path,nombre_archivo)
            print(df)

            df.to_csv(archivo_de_salida, index=False)

In [ ]:
def concatenator():
    path ='./news_urls/el_gato_al_agua_urls'
    e = glob.glob(path + "/*.csv")
    dfs = []
    for i in e:
        dfs.append(pd.read_csv(i))
        
    df_elgato_total = pd.concat(dfs, axis=0)
    df_elgato = df_elgato_total[df_elgato_total['Status'] != 'sin resultados']
    
    return df_elgato

In [ ]:
def scrap_news_function(df_elgato):
    df_elgato = df_elgato
    options = webdriver.ChromeOptions()
    options.add_argument('disable-infobars')
    links = df_elgato['link'].tolist()
    print(links)
    df_list = []
    for e in links:
        link = e
        driver = webdriver.Chrome(executable_path="../DFGM/drivers/chromedriver")
        driver.get(link)
        time.sleep(1)
        
        #Extracción del texto
        texto = driver.find_element_by_xpath('//*[@id="primary"]/div').text
        texto = texto.split('@')[0]
        texto = texto.replace('\n', ' ')
            
        #Extracción del titular
        titular = driver.find_element_by_css_selector('.entry-title').text
        
        date = driver.find_element_by_css_selector('.entry-date').text
        
        autor = driver.find_element_by_css_selector('.entry-author').text
        
        elgato_dict = {'newspaper': 'eltorotv.com', 'link': link,'title': titular, 'author': autor, 'date': date, 'text': texto}
        
        df_elgato_ = pd.DataFrame(elgato_dict, index=[0])
        
        df_elgato_['text'] = df_elgato_['text'].str.replace('\w.*\d+.', '')  
        
        driver.close()
        
        df_list.append(df_elgato_)
        
    df_final_gato = pd.concat(df_list)
    
    df_ = df_elgato.merge(df_final_gato, how='left')
        
    df_ = df_[['keyword', 'newspaper','link', 'title', 'author', 'date', 'text']]
    
    
    return df_

In [ ]:
def save_function(df_to_save):
    today = str(date.today())
    nombre_archivo = 'noticias_elgato_' + today + '.csv'
    path = '../DFGM/DATA_news'
    archivo_de_salida = os.path.join(path,nombre_archivo)
    df_to_save.to_csv(archivo_de_salida, index=False)

In [ ]:
def el_gato_pipeline(lista_keywords):
    
    starttime = datetime.datetime.now()
    url_extractor(lista_keywords)
    df_elgato = concatenator()
    
    df_to_save = scrap_news_function(df_elgato)
    save_function(df_to_save)

    endtime = datetime.datetime.now()
    duration = endtime - starttime
    
    print(duration)
    return df_to_save

In [ ]:
df = el_gato_pipeline(lista_keywords)